<a href="https://colab.research.google.com/github/xi1224-pan/ml-1/blob/main/%E8%AE%8A%E5%BD%A2%E9%87%91%E5%89%9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install transformers


In [13]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_name = "uer/roberta-base-finetuned-jd-binary-chinese"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)



In [14]:
def predict_sentiment(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        label = torch.argmax(probs).item()
    return "正面" if label == 1 else "負面"



In [17]:
print(predict_sentiment("烤大腸很讚，味道都還不錯牛肉也嫩，還有少見的大阪燒，上菜很快技術好"))
print(predict_sentiment("料理，串燒，非常道地這家店"))
print(predict_sentiment("大阪燒260卻沒什麼料，連麵粉、高麗菜都少，薄薄的一份還沒有煎熟，真的很令人失望。很多串燒都是用煎的，連杯水都沒有還要服務費，真不知道好評價是哪裡來的"))


正面
正面
負面
